## Johdanto

Tässä oppitunnissa käsitellään:
- Mitä funktiokutsut ovat ja mihin niitä käytetään
- Miten luodaan funktiokutsu OpenAI:n avulla
- Miten funktiokutsu integroidaan sovellukseen

## Oppimistavoitteet

Tämän oppitunnin jälkeen osaat ja ymmärrät:

- Miksi funktiokutsuja käytetään
- Miten otetaan käyttöön funktiokutsu OpenAI-palvelussa
- Miten suunnitellaan toimivia funktiokutsuja oman sovelluksen tarpeisiin


## Ymmärrä funktiokutsut

Tässä oppitunnissa haluamme rakentaa ominaisuuden koulutus-startupillemme, jonka avulla käyttäjät voivat käyttää chatbotia löytääkseen teknisiä kursseja. Suosittelemme kursseja, jotka sopivat heidän taitotasoonsa, nykyiseen rooliinsa ja kiinnostuksen kohteena olevaan teknologiaan.

Tämän toteuttamiseksi käytämme yhdistelmää seuraavista:
 - `OpenAI` luomaan käyttäjälle keskustelukokemuksen
 - `Microsoft Learn Catalog API` auttamaan käyttäjiä löytämään kursseja heidän pyyntönsä perusteella
 - `Function Calling` ottamaan käyttäjän kyselyn ja lähettämään sen funktiolle API-pyynnön tekemistä varten

Aloitetaan katsomalla, miksi ylipäätään haluaisimme käyttää funktiokutsuja:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # hae uusi vastaus GPT:ltä, jossa se näkee funktion vastauksen


print(second_response.choices[0].message)


### Miksi Function Calling

Jos olet käynyt läpi jonkin muun oppitunnin tässä kurssissa, ymmärrät todennäköisesti, kuinka tehokkaita Large Language Modelit (LLM:t) voivat olla. Toivottavasti olet myös huomannut niiden rajoituksia.

Function Calling on OpenAI-palvelun ominaisuus, joka on suunniteltu ratkaisemaan seuraavat haasteet:

Epäyhtenäinen vastausmuotoilu:
- Ennen function callingia LLM:n vastaukset olivat jäsentymättömiä ja epäyhtenäisiä. Kehittäjien piti kirjoittaa monimutkaista tarkistuskoodia käsitelläkseen erilaisia vastausmuotoja.

Rajoitettu integraatio ulkoiseen dataan:
- Ennen tätä ominaisuutta oli hankalaa tuoda sovelluksen muista osista saatavaa dataa keskusteluun mukaan.

Vakiinnuttamalla vastausmuodot ja mahdollistamalla sujuvan integraation ulkoisen datan kanssa, function calling helpottaa kehitystyötä ja vähentää lisävalidoinnin tarvetta.

Käyttäjät eivät voineet saada vastauksia kuten "Mikä on tämänhetkinen sää Tukholmassa?". Tämä johtui siitä, että mallit olivat rajoittuneet siihen aikaan, jolloin niiden data oli koulutettu.

Katsotaanpa alla olevaa esimerkkiä, joka havainnollistaa tätä ongelmaa:

Oletetaan, että haluamme luoda tietokannan opiskelijoiden tiedoista, jotta voimme suositella heille sopivaa kurssia. Alla on kaksi opiskelijakuvausta, jotka sisältävät hyvin samankaltaista tietoa.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Haluamme lähettää tämän LLM:lle, jotta se voi jäsentää tiedot. Tätä voidaan myöhemmin käyttää sovelluksessamme, kun lähetämme tietoja API:lle tai tallennamme ne tietokantaan.

Luodaan kaksi identtistä kehotetta, joilla ohjeistamme LLM:ää siitä, mitä tietoja olemme kiinnostuneita saamaan:


Haluamme lähettää tämän LLM:lle, jotta se voi jäsentää tuotteellemme tärkeät osat. Näin voimme luoda kaksi identtistä kehotetta ohjeistamaan LLM:ää:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Kun olemme luoneet nämä kaksi kehotetta, lähetämme ne LLM:lle käyttämällä `openai.ChatCompletion`. Tallennamme kehotteen `messages`-muuttujaan ja määritämme rooliksi `user`. Tämä jäljittelee käyttäjän viestin kirjoittamista chatbotille.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Nyt voimme lähettää molemmat pyynnöt LLM:lle ja tarkastella saamaamme vastausta.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Vaikka kehotteet ovat samat ja kuvaukset muistuttavat toisiaan, voimme saada `Grades`-ominaisuudesta erilaisia muotoja.

Jos suoritat yllä olevan solun useita kertoja, muoto voi olla joko `3.7` tai `3.7 GPA`.

Tämä johtuu siitä, että LLM käsittelee jäsentymätöntä dataa kirjoitetun kehotteen muodossa ja palauttaa myös jäsentymätöntä dataa. Meidän täytyy käyttää jäsenneltyä muotoa, jotta tiedämme, mitä odottaa, kun tallennamme tai käytämme tätä dataa.

Käyttämällä funktionaalista kutsua voimme varmistaa, että saamme takaisin jäsenneltyä dataa. Kun käytämme funktiokutsua, LLM ei oikeasti kutsu tai suorita mitään funktioita. Sen sijaan luomme rakenteen, jota LLM noudattaa vastauksissaan. Käytämme näitä jäsenneltyjä vastauksia tietääksemme, mitä funktiota sovelluksissamme tulee käyttää.


![Funktiokutsun kulkukaavio](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.fi.png)


### Käyttötapaukset funktiokutsuille

**Ulkoisten työkalujen käyttäminen**  
Chatbotit ovat erinomaisia vastaamaan käyttäjien kysymyksiin. Funktiokutsujen avulla chatbotit voivat käyttää käyttäjien viestejä tiettyjen tehtävien suorittamiseen. Esimerkiksi opiskelija voi pyytää chatbotia: "Lähetä sähköposti opettajalleni ja kerro, että tarvitsen lisää apua tässä aiheessa." Tämä voi tehdä funktiokutsun `send_email(to: string, body: string)`

**API- tai tietokantakyselyiden luominen**  
Käyttäjät voivat etsiä tietoa luonnollisella kielellä, joka muunnetaan muotoiltuun kyselyyn tai API-pyyntöön. Esimerkkinä opettaja voi kysyä: "Ketkä opiskelijat suorittivat viimeisimmän tehtävän?" ja tämä voisi kutsua funktiota nimeltä `get_completed(student_name: string, assignment: int, current_status: string)`

**Rakenteisen datan luominen**  
Käyttäjät voivat ottaa tekstikappaleen tai CSV-tiedoston ja käyttää LLM:ää tärkeän tiedon poimimiseen siitä. Esimerkiksi opiskelija voi muuntaa Wikipedia-artikkelin rauhansopimuksista tekoälymuistikorteiksi. Tämä onnistuu käyttämällä funktiota `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Ensimmäisen funktiokutsun luominen

Funktiokutsun luominen koostuu kolmesta päävaiheesta:
1. Kutsutaan Chat Completions APIa, annetaan lista funktioista ja käyttäjän viesti
2. Luetaan mallin vastaus ja suoritetaan tarvittava toiminto, esimerkiksi funktio- tai API-kutsu
3. Tehdään uusi kutsu Chat Completions APIin, annetaan funktiosta saatu vastaus, jotta mallilla on tarvittavat tiedot käyttäjälle vastaamiseen


![Flow of a Function Call](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.fi.png)


### Funktion kutsun osat

#### Käyttäjän syöte

Ensimmäinen vaihe on luoda käyttäjän viesti. Tämän voi määrittää dynaamisesti ottamalla arvon tekstikentästä tai voit asettaa arvon suoraan tähän. Jos työskentelet Chat Completions API:n kanssa ensimmäistä kertaa, meidän täytyy määritellä viestin `role` ja `content`.

`role` voi olla joko `system` (sääntöjen luominen), `assistant` (malli) tai `user` (loppukäyttäjä). Funktiokutsua varten asetamme tämän arvoksi `user` ja annamme esimerkkikysymyksen.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funktioiden luominen.

Seuraavaksi määrittelemme funktion ja sen parametrit. Käytämme tässä vain yhtä funktiota nimeltä `search_courses`, mutta voit luoda useita eri funktioita.

**Tärkeää**: Funktiot lisätään järjestelmäviestiin LLM:lle ja ne lasketaan mukaan käytettävissä oleviin tokeneihin.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Määritelmät**

Funktiomäärittelyn rakenne koostuu useista tasoista, joilla jokaisella on omat ominaisuutensa. Tässä on yhteenveto sisäkkäisestä rakenteesta:

**Funktion ylimmän tason ominaisuudet:**

`name` - Funktion nimi, joka halutaan kutsua.

`description` - Kuvaus siitä, miten funktio toimii. Tässä on tärkeää olla tarkka ja selkeä.

`parameters` - Lista arvoista ja muodosta, jotka halutaan mallin tuottavan vastauksessaan.

**Parametriobjektin ominaisuudet:**

`type` - Parametriobjektin tietotyyppi (yleensä "object")

`properties` - Luettelo niistä arvoista, joita malli käyttää vastauksessaan

**Yksittäisen parametrin ominaisuudet:**

`name` - Määritellään epäsuorasti avaimen perusteella (esim. "role", "product", "level")

`type` - Tämän tietyn parametrin tietotyyppi (esim. "string", "number", "boolean")

`description` - Kuvaus kyseisestä parametrista

**Valinnaiset ominaisuudet:**

`required` - Taulukko, jossa luetellaan, mitkä parametrit ovat pakollisia, jotta funktiokutsu voidaan suorittaa loppuun


### Funktiokutsun tekeminen
Kun funktio on määritelty, meidän täytyy nyt liittää se Chat Completion API -kutsuun. Tämä tehdään lisäämällä `functions` pyyntöön. Tässä tapauksessa `functions=functions`.

Lisäksi on mahdollista asettaa `function_call` arvoksi `auto`. Tämä tarkoittaa, että annamme LLM:n päättää, mitä funktiota tulisi kutsua käyttäjän viestin perusteella sen sijaan, että määrittelisimme sen itse.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nyt tarkastellaan vastausta ja sen muotoilua:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Voit huomata, että funktion nimi on kutsuttu ja käyttäjän viestistä LLM pystyi löytämään tiedot, jotka sopivat funktion argumentteihin.


## 3. Funktiokutsujen integrointi sovellukseen.

Kun olemme testanneet LLM:n muotoillun vastauksen, voimme nyt integroida sen sovellukseen.

### Prosessin hallinta

Jotta voimme integroida tämän sovellukseemme, tehdään seuraavat vaiheet:

Ensin tehdään kutsu OpenAI-palveluihin ja tallennetaan viesti muuttujaan nimeltä `response_message`.


In [ ]:
response_message = response.choices[0].message

Nyt määrittelemme funktion, joka kutsuu Microsoft Learn API:a saadakseen luettelon kursseista:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Hyvänä käytäntönä tarkistamme ensin, haluaako malli kutsua funktiota. Tämän jälkeen luomme jonkin saatavilla olevista funktioista ja yhdistämme sen kutsuttavaan funktioon.
Seuraavaksi otamme funktion argumentit ja yhdistämme ne LLM:n antamiin argumentteihin.

Lopuksi liitämme funktiokutsun viestin ja arvot, jotka `search_courses`-viesti palautti. Näin LLM:llä on kaikki tarvittava tieto, jotta se voi vastata käyttäjälle luonnollisella kielellä.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kooditehtävä

Hienoa työtä! Jatkaaksesi OpenAI Function Calling -osaamisesi kehittämistä voit rakentaa: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Lisää funktion parametreja, jotka voivat auttaa oppijoita löytämään lisää kursseja. Löydät saatavilla olevat API-parametrit täältä:
 - Luo toinen funktiokutsu, joka ottaa oppijalta enemmän tietoa, kuten heidän äidinkielensä
 - Toteuta virheenkäsittely, jos funktiokutsu ja/tai API-kutsu ei palauta sopivia kursseja



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta huomioithan, että automaattiset käännökset saattavat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisissä tapauksissa suosittelemme ammattimaisen ihmiskääntäjän käyttöä. Emme ole vastuussa tämän käännöksen käytöstä mahdollisesti aiheutuvista väärinkäsityksistä tai tulkinnoista.
